### This notebook aims at providing a pipeline for a binary CNN classifier but can be used for any model of the keras library.

In [1]:
import os
from os import listdir
import numpy as np
import random
from PIL import Image, ImageOps
from tensorflow import keras

In [2]:
#Please change the directories to the one where you downloaded the datasets
train_dir = "C:/Users/thoma/OneDrive/Desktop/Centrale/3A/Spé info/SID/Méthodo/Projet/training"
val_dir = "C:/Users/thoma/OneDrive/Desktop/Centrale/3A/Spé info/SID/Méthodo/Projet/validation"

In [3]:
#We resize the images to a 256x256 format
def resize_image(img):
    res = img.resize((256,256),Image.BICUBIC)
    return res

In [4]:
class DataLoader:
    
    def __init__(self, train_dir, val_dir):
        self.train_dir = train_dir
        self.val_dir = val_dir
        
    def load_data(self):
        
        Training_img = []
        
        Dmg_train_dir = self.train_dir + "/00-damage"
        
        for images in os.listdir(Dmg_train_dir):
            im = Image.open(Dmg_train_dir + '/' + images)
            Training_img.append([im,1])
            
        Whole_train_dir = self.train_dir + "/01-whole"
        
        for images in os.listdir(Whole_train_dir):
            im = Image.open(Whole_train_dir + '/' + images)
            Training_img.append([im,0])
            
        random.shuffle(Training_img)
        
        #We needed to take images and labels as a whole as to shuffle them so as to avoid any bias when training models
        
        X_train,y_train = [],[]
        
        for k in Training_img:
            X_train.append(k[0])
            y_train.append(k[1])
        
        X_train_keras = np.array([np.asarray(resize_image(img)) for img in X_train])
        y_train_keras = np.array(y_train)
        
        Validation_img = []
        
        Dmg_val_dir = self.val_dir + "/00-damage"
        
        for images in os.listdir(Dmg_val_dir):
            im = Image.open(Dmg_val_dir + '/' + images)
            Validation_img.append([im,1])
            
        Whole_val_dir = self.val_dir + "/01-whole"
        
        for images in os.listdir(Whole_val_dir):
            im = Image.open(Whole_val_dir + '/' + images)
            Validation_img.append([im,0])
            
        random.shuffle(Validation_img)
        
        #Same reason as above for this part of the code
        
        X_val,y_val = [],[]
        
        for k in Validation_img:
            X_val.append(k[0])
            y_val.append(k[1])
            
        X_val_keras = np.array([np.asarray(resize_image(img)) for img in X_val])
        y_val_keras = np.array(y_val)
        
        return X_train_keras, y_train_keras, X_val_keras, y_val_keras                        

We chose a LeNet-5 architecture of CNN for two reasons:

 First, we think it is a nice reference to the work of a great french researcher, Mr Yann LeCun. 
 Second, since it was one the first CNN model it is surely not the most efficient to date and thus we can have comparison with more classical ML models (less designated for object detection than CNNs) 

In [5]:
# Build the model
model = keras.Sequential([
    keras.layers.Conv2D(6, (5, 5), padding='same', input_shape=(256, 256, 3)),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    keras.layers.Conv2D(16, (5, 5)),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(120),
    keras.layers.Activation('relu'),
    keras.layers.Dense(84),
    keras.layers.Activation('relu'),
    keras.layers.Dense(1),
    keras.layers.Activation('sigmoid')
])

In [6]:
# Compile the model with a binary cross entropy since we only have two categories (damaged and whole)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [7]:
#Fit the model
X_train, y_train, X_val, y_val = DataLoader(train_dir, val_dir).load_data()
model.fit(X_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
58/58 [==============================] - 27s 446ms/step - loss: 32.7628 - accuracy: 0.6188
Epoch 2/5
58/58 [==============================] - 26s 455ms/step - loss: 0.3453 - accuracy: 0.8657
Epoch 3/5
58/58 [==============================] - 25s 437ms/step - loss: 0.1272 - accuracy: 0.9636
Epoch 4/5
58/58 [==============================] - 24s 422ms/step - loss: 0.0404 - accuracy: 0.9918
Epoch 5/5
58/58 [==============================] - 25s 428ms/step - loss: 0.0261 - accuracy: 0.9940


In [8]:
results = model.evaluate(X_val, y_val, batch_size=32)
print("test loss, test acc:", results)

15/15 [==============================] - 1s 74ms/step - loss: 1.3246 - accuracy: 0.7174
test loss, test acc: [1.3245657682418823, 0.717391312122345]
